In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [1]:


# Load the datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

# Merge transactions with products to get product category and price
transactions = pd.merge(transactions, products, on='ProductID')

# Feature Engineering: Create customer features
# 1. Total spending per customer
total_spending = transactions.groupby('CustomerID')['TotalValue'].sum().reset_index()
total_spending.columns = ['CustomerID', 'TotalSpending']

# 2. Average transaction value per customer
avg_transaction_value = transactions.groupby('CustomerID')['TotalValue'].mean().reset_index()
avg_transaction_value.columns = ['CustomerID', 'AvgTransactionValue']

# 3. Favorite product category (most purchased category)
favorite_category = transactions.groupby(['CustomerID', 'Category']).size().reset_index(name='Count')
favorite_category = favorite_category.loc[favorite_category.groupby('CustomerID')['Count'].idxmax()]
favorite_category = favorite_category[['CustomerID', 'Category']]
favorite_category.columns = ['CustomerID', 'FavoriteCategory']

# 4. Number of transactions per customer
num_transactions = transactions.groupby('CustomerID').size().reset_index(name='NumTransactions')

# 5. Total quantity purchased per customer
total_quantity = transactions.groupby('CustomerID')['Quantity'].sum().reset_index()
total_quantity.columns = ['CustomerID', 'TotalQuantity']

# Merge all features into a single customer feature dataframe
customer_features = pd.merge(customers, total_spending, on='CustomerID', how='left')
customer_features = pd.merge(customer_features, avg_transaction_value, on='CustomerID', how='left')
customer_features = pd.merge(customer_features, favorite_category, on='CustomerID', how='left')
customer_features = pd.merge(customer_features, num_transactions, on='CustomerID', how='left')
customer_features = pd.merge(customer_features, total_quantity, on='CustomerID', how='left')

# Handle missing values (if any)
customer_features.fillna(0, inplace=True)

# Encode categorical features (FavoriteCategory and Region)
customer_features = pd.get_dummies(customer_features, columns=['FavoriteCategory', 'Region'], drop_first=True)

# Normalize numerical features
scaler = StandardScaler()
numerical_features = ['TotalSpending', 'AvgTransactionValue', 'NumTransactions', 'TotalQuantity']
customer_features[numerical_features] = scaler.fit_transform(customer_features[numerical_features])

# Drop non-relevant columns for similarity calculation
customer_features.set_index('CustomerID', inplace=True)
customer_features.drop(columns=['CustomerName', 'SignupDate'], inplace=True)

# Calculate cosine similarity between customers
similarity_matrix = cosine_similarity(customer_features)

# Convert similarity matrix to a dataframe
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features.index, columns=customer_features.index)

# Function to get top 3 similar customers
def get_top_similar_customers(customer_id, similarity_df, top_n=3):
    # Exclude the customer itself (similarity score = 1)
    similar_customers = similarity_df[customer_id].drop(customer_id).sort_values(ascending=False).head(top_n)
    return similar_customers

# Generate recommendations for the first 20 customers (C0001 - C0020)
lookalike_map = {}
for customer_id in customer_features.index[:20]:
    similar_customers = get_top_similar_customers(customer_id, similarity_df)
    # Convert numpy.float64 to native Python float for better CSV compatibility
    lookalike_map[customer_id] = [[similar_customer, float(score)] for similar_customer, score in zip(similar_customers.index, similar_customers.values)]

# Convert the map to a dataframe
lookalike_df = pd.DataFrame(lookalike_map.items(), columns=['CustomerID', 'Lookalikes'])

# Save the lookalike recommendations to a CSV file
lookalike_df.to_csv('Ankur_Bhadauria_Lookalike.csv', index=False)

# Print the lookalike recommendations
print(lookalike_df)

444
